In [1]:
import pandas as pd
import os

def find_bug(df):
    """
    df_offers имеет странные дублирующиеся model_id_pred. Векроятно это баг в функции top_n_similar 
    (не находит достаточное количество до n шт матчей + дальнейший convert_columns_to_int)
    """
    columns_to_check = ['model_id_pred_1', 'model_id_pred_2', 'model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
    duplicate_rows_indices = df[df[columns_to_check].apply(lambda row: row.duplicated().sum() >= 1, axis=1)].index
    new_df = df.loc[duplicate_rows_indices]
    return new_df

def merge_columns_to_list(df, n, column_name):
    """
    Функция для объединения значений из нескольких колонок в один список.
    
    Параметры:
        df (DataFrame): Исходный DataFrame.
        n (int): Количество колонок, которые нужно объединить.
        column_name (str): Имя обрабатываемых колонок.
        
    Возвращает:
        DataFrame: DataFrame с добавленным столбцом, содержащим объединенные значения в виде списка.
    """
    # Создаем список колонок для объединения
    columns = [f"{column_name}_{i}" for i in range(1, n+1)]
    
    # Объединяем значения в список
    df[column_name] = df[columns].apply(lambda row: row.tolist(), axis=1)
    
    return df

def exclude_etalon_from_df(df, df_etalon, imya_kategorii):
    df_exclude = df_etalon[df_etalon.category_name == imya_kategorii]
    print(f"{imya_kategorii}: exclude {len(df_exclude)} rows")
    return df[~df.name.isin(df_exclude.name)]

pth_models = "/home/sondors/Documents/price/BERT_data/data/27-03-2024_Timofey/740101_models.csv"
pth_offers = "/home/sondors/Documents/price/ColBERT_data/10_categories/740101/test/740101_lr04_bsize230_offers_top_n.csv"
# pth_csv_exclude = '/home/sondors/Documents/price/offers_clusterization/data/14_categories_balanced.csv'

# id_category = {
    #     3902: 'диктофоны, портативные рекордеры',
    #     510402: 'электронные книги',
    #     4302: 'автомобильные телевизоры, мониторы',
    #     2815: 'смарт-часы и браслеты',
    #     3901: 'портативные медиаплееры',
    #     3904: 'портативная акустика',
    #     2801: 'мобильные телефоны',
    #     3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    #     510401: 'планшетные компьютеры и мини-планшеты',
    #     2102: 'наушники, гарнитуры, наушники c микрофоном',
    #     3903: 'радиоприемники, радиобудильники, радиочасы',
    #     3907: 'магнитолы',
    #     280801: 'GPS-навигаторы'
    #     }

id_category = {
    # 921201: 'корм для кошек',
    # 963401: 'колбасы',
    # 710502: 'стиральные машины',
    # 977135: 'ботинки, полуботинки',
    # 911906: 'парфюмерия',
    # 976132: 'туфли и лоферы',
    740101: 'кухонные мойки',
    # 7106: 'микроволновые печи',
    # 921401: 'корм для собак',
    # 963302: 'пицца'
    }

df_models = pd.read_csv(pth_models, sep=";")
# df_models = df_models.drop(columns=['average_price', 'comment'])
df_offers = pd.read_csv(pth_offers, sep=";")
# df_offers = df_offers.drop(columns=['true_match', 'false_match'])

# df_exclude = pd.read_csv(pth_csv_exclude, sep=';')
# for category in id_category.values():
#     df_new = exclude_etalon_from_df(df_offers, df_exclude, category)

df_offers = merge_columns_to_list(df_offers, 5, "model_id_pred")
df_offers = merge_columns_to_list(df_offers, 5, "similarity")
df_offers


,name,model_id,offer_id,modelized_by_barcode,category_name,category_id,price,attrs,description,external_category,...,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_pred,similarity
0,втехнике.рф Кухонная мойка Omoikiri Daisen 78-...,1009167,1676995013,False,кухонные мойки,740101,18688.0,NaN,NaN,Сантехника :: Кухонные мойки :: Гранитные (иск...,...,1832466,27.17,4165635,26.55,1832442,25.05,5144198,24.78,"[1009167, 1832466, 4165635, 1832442, 5144198]","[29.34, 27.17, 26.55, 25.05, 24.78]"
1,"Кухонная мойка Florentina Вега 360, бежевый",737066,1655080546,False,кухонные мойки,740101,8300.0,"[{'id': 9167, 'type': 'b', 'value': 1.0}]",МОЙКА ДЛЯ КУХНИ ИЗ КАМНЯ FLORENTINA ВЕГА 360 б...,Мойки,...,728528,23.73,737063,22.56,5143739,21.83,521500,19.66,"[737066, 728528, 737063, 5143739, 521500]","[30.27, 23.73, 22.56, 21.83, 19.66]"
2,Кухонная мойка VIGRO VG101 терра,5769564,1736132544,True,кухонные мойки,740101,6854.0,"[{'id': 9171, 'type': 'b', 'value': 1.0}]",Кухонная мойка из искусственного камня Vigro V...,Строительство и ремонт :: Сантехника :: Мойки ...,...,5144312,23.20,520177,21.31,902538,21.25,520175,21.23,"[5769564, 5144312, 520177, 902538, 520175]","[29.28, 23.2, 21.31, 21.25, 21.23]"
3,Мойка для кухни Franke Antea AZG 611-62 (114.0...,5143759,1793522446,True,кухонные мойки,740101,14990.0,"[{'id': 9167, 'type': 'b', 'value': 1.0}]",Основные характеристики - Тип монтажа: врезной...,Для кухни :: Мойки для кухни,...,519324,23.92,1538790,23.44,949441,21.50,5143760,20.89,"[5143759, 519324, 1538790, 949441, 5143760]","[26.84, 23.92, 23.44, 21.5, 20.89]"
4,"Мойка гранитная платина 54 см, Omoikiri, Tedor...",5144221,1276060286,False,кухонные мойки,740101,31888.0,"[{'id': 9175, 'type': 'b', 'value': 1.0}, {'id...","Высота, мм: 200 | Ширина, мм: 550 | Глубина, м...",Кухонные мойки :: Гранитные мойки,...,1832511,24.52,5144256,23.31,5144230,22.97,5144263,22.56,"[5144221, 1832511, 5144256, 5144230, 5144263]","[28.23, 24.52, 23.31, 22.97, 22.56]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12509,втехнике.рф Кухонная мойка Dr.Gans Габи террак...,728530,1676982536,False,кухонные мойки,740101,10701.0,NaN,NaN,Сантехника :: Кухонные мойки :: Гранитные (иск...,...,5143743,19.95,728531,19.59,1538773,19.42,5143958,16.33,"[728530, 5143743, 728531, 1538773, 5143958]","[23.17, 19.95, 19.59, 19.42, 16.33]"
12510,Кухонная мойка Omoikiri Tadzava 44-U-IN Quadro,1832528,1437177109,False,кухонные мойки,740101,20888.0,"[{'id': 9175, 'type': 'b', 'value': 1.0}]",Мойка из высококачественной нержавеющей стали ...,Рекомендуем,...,6300504,28.44,1832521,25.48,728565,21.34,1832437,21.23,"[1832528, 6300504, 1832521, 728565, 1832437]","[30.55, 28.44, 25.48, 21.34, 21.23]"
12511,Мойка Bamboo Кардинал 570 дюна 29.030.D0570.40...,5143683,1432653107,False,кухонные мойки,740101,9361.0,NaN,NaN,Кухонные мойки :: Мойки из композитных материалов,...,5143751,19.59,536592,17.39,536500,17.23,5143685,16.70,"[5143683, 5143751, 536592, 536500, 5143685]","[26.8, 19.59, 17.39, 17.23, 16.7]"
12512,Кухонная мойка Lex Garda 620 Chocolate,3601776,1813071406,False,кухонные мойки,740101,6490.0,NaN,NaN,Покупки для кухни :: Встраиваемая техника для ...,...,7794818,22.78,942333,20.88,620870,20.56,5143689,20.55,"[3601776, 7794818, 942333, 620870, 5143689]","[28.17, 22.78, 20.88, 20.56, 20.55]"


In [2]:
def accuracy_top_n(offer_model_id, offer_model_id_pred, top_k):
    correct_predictions = 0

    for true_id, predicted_ids in zip(offer_model_id, offer_model_id_pred):
        top_k_predictions = predicted_ids[:top_k]
        if true_id in top_k_predictions:
            correct_predictions += 1

    accuracy = correct_predictions / len(offer_model_id)
    return accuracy

def calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k): # проверить функцию
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for true_id, predicted_ids in zip(offer_model_id, offer_model_id_pred):
        top_k_predictions = predicted_ids[:top_k]
        
        if true_id in top_k_predictions:
            true_positives += 1
        else:
            false_negatives += 1
            false_positives += (true_id in predicted_ids)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return precision, recall

def calc_acc_for_category(df_offers, df_models, imya_kategorii):
    print(f"{imya_kategorii}")
    df_offers = df_offers[df_offers.category_name == imya_kategorii]
    df_models = df_models[df_models.category_name == imya_kategorii]

    offer_names = list(df_offers['name'])
    offer_model_id = list(df_offers['model_id'])
    offer_model_id_pred = list(df_offers['model_id_pred'])

    model_full_names = list(df_models['full_name'])

    print(f"\tофферов {len(offer_model_id)} шт поиск матча для каждого среди {len(model_full_names)} шт моделей:\n")

    top1_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 1)
    top2_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 2)
    top3_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 3)
    top4_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 4)
    top5_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 5)

    top1_precision, top1_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=1)
    top3_precision, top3_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=3)
    top5_precision, top5_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=5)

    print(f'\tTop-1: Accuracy = {top1_accuracy:.2%}')
    print(f'\tTop-3: Accuracy = {top3_accuracy:.2%}')
    print(f'\tTop-5: Accuracy = {top5_accuracy:.2%}\n')

    # print(f'\tTop-1: Accuracy = {top1_accuracy:.2%}, Precision = {top1_precision:.2%}, Recall = {top1_recall:.2%}')
    # print(f'\tTop-3: Accuracy = {top3_accuracy:.2%}, Precision = {top3_precision:.2%}, Recall = {top3_recall:.2%}')
    # print(f'\tTop-5: Accuracy = {top5_accuracy:.2%}, Precision = {top5_precision:.2%}, Recall = {top5_recall:.2%}\n')
    return (top1_accuracy, top2_accuracy, top3_accuracy, top4_accuracy, top5_accuracy)

for category in id_category.values():
    accuracies_top_n = calc_acc_for_category(df_offers, df_models, category)

кухонные мойки
	офферов 12514 шт поиск матча для каждого среди 9314 шт моделей:

	Top-1: Accuracy = 95.29%
	Top-3: Accuracy = 98.24%
	Top-5: Accuracy = 98.62%



In [3]:
def write_errors(df_offers, df, df_models, dst_fld, id_category):
    os.makedirs(dst_fld, exist_ok=True)
    for category in id_category.values():
        category_error = df[df.category_name == category]
        category_all = df_offers[df_offers.category_name == category]
        accuracies_top_n = calc_acc_for_category(category_error, df_models, category)
        with open(os.path.join(dst_fld, f"{category}.txt"), "a") as txt:
            txt.write(f'Всего офферов {len(category_all)} шт из них ошибок {len(category_error)} шт\nРаспределение ошибок:\n')
            txt.write(f'\nTop-1: Accuracy = {accuracies_top_n[0]:.2%}')
            txt.write(f'\nTop-2: Accuracy = {accuracies_top_n[1]:.2%}')
            txt.write(f'\nTop-3: Accuracy = {accuracies_top_n[2]:.2%}')
            txt.write(f'\nTop-4: Accuracy = {accuracies_top_n[3]:.2%}')
            txt.write(f'\nTop-5: Accuracy = {accuracies_top_n[4]:.2%}\n\n')
            txt.write("Истинная модель:\nПоле name оффера --> Найденная модель топ1")
            txt.write("\n____________________________________________________________\n")
            for index, row in category_error.iterrows():
                model_gt = list(df_models[df_models.model_id == row['model_id']]['full_name'])[0]
                model_pred = list(df_models[df_models.model_id == row['model_id_pred_1']]['full_name'])[0]
                txt.write(f"\n{model_gt}:\n{row['name']} --> {model_pred}\n")                
def exclude_from_df(df, col):
    return df[df['model_id'] != df[col]].reset_index(drop=True)

clms = ['model_id_pred_1','model_id_pred_2','model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
df = df_offers.copy()
for i in range(len(clms)):
    df = exclude_from_df(df, clms[i])
    dst_fld = f"/home/sondors/Documents/price/ColBERT/EVAL/errors_ColBERT/exclude_top_{i+1}"
    write_errors(df_offers, df, df_models, dst_fld, id_category)

кухонные мойки
	офферов 590 шт поиск матча для каждого среди 9314 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 62.71%
	Top-5: Accuracy = 70.68%

кухонные мойки
	офферов 287 шт поиск матча для каждого среди 9314 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 23.34%
	Top-5: Accuracy = 39.72%

кухонные мойки
	офферов 220 шт поиск матча для каждого среди 9314 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 0.00%
	Top-5: Accuracy = 21.36%

кухонные мойки
	офферов 191 шт поиск матча для каждого среди 9314 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 0.00%
	Top-5: Accuracy = 9.42%

кухонные мойки
	офферов 173 шт поиск матча для каждого среди 9314 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 0.00%
	Top-5: Accuracy = 0.00%



In [4]:
df

,name,model_id,offer_id,modelized_by_barcode,category_name,category_id,price,attrs,description,external_category,...,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_pred,similarity
0,Кухонная мойка Franke MARIS (127.0525.273),5143789,1749333099,False,кухонные мойки,740101,51690.0,"[{'id': 9168, 'type': 'b', 'value': 1.0}]",Мойка для кухни швейцарского производства из н...,Сантехника :: Кухонные мойки,...,851810,19.86,6300439,19.80,518095,19.41,851811,19.33,"[5718993, 851810, 6300439, 518095, 851811]","[19.97, 19.86, 19.8, 19.41, 19.33]"
1,"Врезная кухонная мойка 42 см, Omoikiri Tedori ...",1832485,1832862939,False,кухонные мойки,740101,27600.0,"[{'id': 9168, 'type': 'b', 'value': 1.0}]",• Технология производства «Tetogranit» заключа...,Мойки,...,5144238,25.62,5144235,24.88,5144234,24.86,5144240,24.31,"[5144241, 5144238, 5144235, 5144234, 5144240]","[28.06, 25.62, 24.88, 24.86, 24.31]"
2,Мойка Omoikiri 4993782,707177,1736218599,True,кухонные мойки,740101,62888.0,"[{'id': 9169, 'type': 'b', 'value': 1.0}, {'id...",reviewsCount: 1 | reviewsRating: 5 | cashback:...,Строительство и ремонт :: Сантехника :: Мойки ...,...,1832484,25.91,1832503,25.91,1832511,25.89,1832519,25.88,"[1832487, 1832484, 1832503, 1832511, 1832519]","[26.0, 25.91, 25.91, 25.89, 25.88]"
3,"Врезная кухонная мойка 50.5 см, BERGG ZETT lab...",5147926,1850148582,True,кухонные мойки,740101,7125.0,"[{'id': 9169, 'type': 'b', 'value': 1.0}]","Кухонная мойка Zett Lab Модель 190 60,5x49 T19...",Все товары :: Строительство и ремонт :: Сантех...,...,5147932,24.02,5147928,23.80,5147930,23.58,5147931,23.53,"[5147933, 5147932, 5147928, 5147930, 5147931]","[25.45, 24.02, 23.8, 23.58, 23.53]"
4,"Мойка кухонная MIXLINE 528182, врезная, толщин...",5769443,1807896594,True,кухонные мойки,740101,3173.0,NaN,В ассортименте нашего магазина вы можете найти...,Все товары :: Строительство и ремонт :: Сантех...,...,7794830,26.16,5769437,25.56,7794823,24.64,7794840,24.56,"[6300469, 7794830, 5769437, 7794823, 7794840]","[26.67, 26.16, 25.56, 24.64, 24.56]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Угловая мойка MIXLINE 525205,1538858,1760294279,True,кухонные мойки,740101,8025.0,"[{'id': 9173, 'type': 'b', 'value': 1.0}]","Угловая мойка MIXLINE ml-gm25 двухчашевая, с к...",Сантехника :: Сантехника для кухни :: Мойки дл...,...,5769444,25.33,7794833,24.70,7794847,23.62,7794845,22.97,"[7794826, 5769444, 7794833, 7794847, 7794845]","[25.39, 25.33, 24.7, 23.62, 22.97]"
169,Мойка TOLERO 067302,674883,1736702831,True,кухонные мойки,740101,5800.0,"[{'id': 9170, 'type': 'b', 'value': 1.0}]",Эффектная черная мойка Classic Tolero прекрасн...,Строительство и ремонт :: Сантехника :: Мойки ...,...,521289,23.75,562792,23.64,562793,23.55,674884,23.52,"[521287, 521289, 562792, 562793, 674884]","[23.97, 23.75, 23.64, 23.55, 23.52]"
170,Мойка кухонная Granula 79х50 бежевый матовый,1055174,1869635924,False,кухонные мойки,740101,13145.0,"[{'id': 9167, 'type': 'b', 'value': 1.0}]",NaN,Гранитные мойки,...,519497,22.45,1576525,22.23,1055170,22.22,1055171,22.00,"[519496, 519497, 1576525, 1055170, 1055171]","[24.2, 22.45, 22.23, 22.22, 22.0]"
171,"4997118, кухонная мойка, Omoikiri",6300506,1813173302,True,кухонные мойки,740101,35188.0,NaN,NaN,Все товары :: Строительство и ремонт :: Сантех...,...,5143998,26.28,1832485,26.20,5143997,26.16,5143993,26.12,"[5143994, 5143998, 1832485, 5143997, 5143993]","[26.39, 26.28, 26.2, 26.16, 26.12]"


In [ ]:
# df.to_csv("/home/sondors/Documents/price/ColBERT_data/10_categories/740101/test/740101_model_id_not_in_top_5.csv", sep=';',index=False)